In [ ]:
import os
from os.path import join
import pandas as pd
import numpy as np

import pandas as pd
import seaborn as sns
import numpy as np

import re
from functools import partial

from joblib import load

from tools import enrich_survey

from joblib import load

from matplotlib import ticker
import matplotlib.pyplot as plt

In [ ]:
merged = pd.read_csv("data/shuffled.csv", index_col=0)

In [ ]:
merged = merged.pipe(enrich_survey)

In [ ]:
merged.shape

In [ ]:
# Age
SEX = ['male', 'female']
TOBACCO = ['smoker_current', 'no_smoker']
COMORBITIES = ['no_comorbidity', 'any_comorbidity', 'respiratory', 'cardio-vascular', 'diabetes', 'obesity']
HOSPITALIZED = ['hospitalized','non_hospitalized']
INCLUSION_REASONS = ['samu', 'urgence']
AGE = merged['binned_age'].cat.categories.tolist()

In [ ]:
import statsmodels.api as sm

In [ ]:
from collections import defaultdict
SYMPTOM_DICT = {}
for s in ['tiredness', 'fever', 'cough', 'breathlessness', 'aches',
          'anorexia', 'anosmia', 'ageusia', 'headache', 
#           'upper_respiratory',
          'conjunctivitis']:
    SYMPTOM_DICT[s] = [s]
SYMPTOM_DICT['cutaneous'] = ['rash', 'frostbites']
SYMPTOM_DICT['digestive'] = ['diarrhea', 'vomiting', 'abdo_pain']
SYMPTOM_DICT['cardiopulmonary'] = ['breathlessness', 'chest_opression', 'chest_pain']
for k, v in SYMPTOM_DICT.items():
    merged[k] = np.any(merged[v], axis=1)
SYMPTOMS = list(SYMPTOM_DICT.keys())

In [ ]:
ALL =  ['all']

In [ ]:
obs = {}
from scipy.stats import chisquare
groups = {'Full cohort': ALL, 'Sex': SEX, 'Age': AGE, 'Tobacco usage': TOBACCO, 'Comorbidities': COMORBITIES, 'Symptoms': SYMPTOMS, 'Hospitalized': HOSPITALIZED}

In [ ]:
labels = sum(groups.values(), [])

In [ ]:
merged['constant'] = 1.

In [ ]:
from numpy.linalg import LinAlgError

In [ ]:
sm.families.Binomial()

In [ ]:
X = merged[SEX + TOBACCO + COMORBITIES + AGE + SYMPTOMS + HOSPITALIZED]
y = merged['test_done']

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=10000)
lr.fit(X, y)
p = lr.predict_proba(X)[:, 1]
merged['p_test'] = p
merged.loc[merged['test_done'], 'sample_weight'] = 1 / merged.loc[merged['test_done'], 'p_test']
merged.loc[~merged['test_done'], 'sample_weight'] = 1 / (1 - merged.loc[~merged['test_done'], 'p_test'])
merged['sample_weight'] /= merged.loc[merged['test_done'], 'sample_weight'].sum() / len(merged.loc[merged['test_done']])

In [ ]:
merged['sample_weight'] .describe()

In [ ]:
def make_observation_df(df, target, correct=False):
    negative = df.loc[~(df['pcr_results'].fillna(True))]
    positive = df.loc[df['pcr_results'].fillna(False)]
    obs = {}
    group_obs = {}
    for group_name, group in groups.items():
        in_vc_groups = {}
        for k in group:
            if not correct:
                in_vc = df.loc[df[k], target].value_counts().reindex([True, False]).fillna(0).rename('in').to_frame()
                in_vc_groups[k] = in_vc['in']
                out_vc = df.loc[~df[k], target].value_counts().reindex([True, False]).fillna(0).rename('out').to_frame()
                o = pd.concat((in_vc, out_vc), axis=1)
                o.index = pd.Index(['tested', 'non_tested'])
                obs[k] = o.unstack(0)
            else:
                res = {}
                in_mask = df[k]
                tested_mask = df[target]
                res[('in', 'tested')] = df.loc[in_mask & tested_mask, 'sample_weight'].sum()
                res[('in', 'non_tested')] = df.loc[in_mask & ~(tested_mask.fillna(True)), 'sample_weight'].sum()
                res[('out', 'tested')] = df.loc[~in_mask & tested_mask, 'sample_weight'].sum()
                res[('out', 'non_tested')] = df.loc[~in_mask & ~(tested_mask.fillna(True)), 'sample_weight'].sum()
                obs[k] = pd.Series(res)
            
            if k != 'all':
                y = df[target]
                X = df[[k, 'constant']].astype(float)
                mask = ~y.isna()
                y = y.loc[mask].astype(float)
                X = X.loc[mask]
                sample_weight = df.loc[mask, 'sample_weight']
                try:
                    if correct:
                        logit_mod = sm.GLM(y, X, family=sm.families.Binomial(), var_weights=sample_weight)
                    else:
                        logit_mod = sm.Logit(y, X)
                    logit_res = logit_mod.fit(disp=0)
                    mean = logit_res.params[k]
                    se = logit_res.bse[k]
                    obs[k][('stat', 'odd_ratio_tested_l')], obs[k][('stat', 'odd_ratio_tested')], obs[k][('stat', 'odd_ratio_tested_u')] = np.exp(mean - se), np.exp(mean), np.exp(mean + se)
                    obs[k][('stat', 'p')] = logit_res.pvalues[k]
                except LinAlgError:
                    obs[k][('stat', 'odd_ratio_tested_l')], obs[k][('stat', 'odd_ratio_tested')], obs[k][('stat', 'odd_ratio_tested_u')] = float('inf'), float('inf'), float('inf')
                    obs[k][('stat', 'p')] = 0

            for l in ['in', 'out']:
                obs[k][(l, 'sum')] = obs[k][l].sum()
                obs[k][(l, 'freq_tested')] = obs[k][(l, 'tested')] / obs[k][(l, 'sum')]
                obs[k][(l, 'freq_non_tested')] = obs[k][(l, 'non_tested')] / obs[k][(l, 'sum')]

            for cat, filtered_df in zip(['freq_among_all', 'freq_among_positive', 'freq_among_negative'], [df, positive, negative]):
                try:
                    if correct:
                        obs[k][('count', cat)] = filtered_df.loc[filtered_df[k], 'sample_weight'].sum()
                        total = filtered_df.loc[~filtered_df[k].isna(), 'sample_weight'].sum()
                        obs[k][('stat', cat)] = obs[k][('count', cat)] / total
                    else:
                        obs[k][('stat', cat)] = filtered_df[k].value_counts()[True] / filtered_df[k].count()
                        obs[k][('count', cat)] = filtered_df[k].value_counts()[True]
                except KeyError:
                    obs[k][('stat', cat)] = 0
                    obs[k][('count', cat)] = 0
        if group_name in ['Age', 'Sex']:
            y = df[target]
            # Hack
            X = df[group[:-1] + ['constant']].astype(float)
            mask = ~y.isna()        
            y = y.loc[mask].astype(float)
            X = X.loc[mask]
            sample_weight = df.loc[mask, 'sample_weight']
            try:
                if correct:
                    logit_mod = sm.GLM(y, X, family=sm.families.Binomial(), var_weights=sample_weight)
                else:
                    logit_mod = sm.Logit(y, X)
                d = np.eye(len(group))[:-1, :]
                logit_res = logit_mod.fit(disp=0)
                contrast = logit_res.wald_test(d)
                group_obs[group_name] = {'p': contrast.pvalue}
            except LinAlgError:
                group_obs[group_name] = {'p': 0}
        elif group_name != 'Full cohort':
            group_obs[group_name] = {'p': obs[group[0]][('stat', 'p')]}
    obs = pd.concat(obs.values(), keys=obs.keys()).unstack((1, 2))
    group_obs = pd.DataFrame(group_obs).T
    return obs, group_obs

names = {'tiredness': 'Fatigue', 'cough': 'Cough', 'fever': 'Fever', 'cardiopulmonary': 'cardiolpulmonary', 'anosmia': 'Anosmia', 'anorexia': 'Anorexia', 'ageusia': 'Ageusia', 'aches': 'Myalgia',
        'breathlessness': 'Breathlessness', 'digestive': 'Digestive troubles', 'cutaneous': 'Cutaneous symptoms', 'conjunctivitis': 'Conjunctivitis', 'headache': 'Headache', 'upper_respiratory': 'Upper respiratory symptoms',
        'smoker_current': 'Smoking', 'cardiolpulmonary': 'Cardio-pulmonary symptoms',
        'all': 'Full cohort',
        'obesity': 'Obesity', 'male': 'Male', 'female': 'Female',
        'any_comorbidity': 'Any',
        'cardiopulmonary': 'Cardio-pulmonary',
        'respiratory': 'Respiratory',
        'cardio-vascular': 'Cardio-vascular',
         'no_smoker': 'Not smoking',
         'urgence': 'Emergency',
         'samu': 'Emergency call',
         'no_comorbidity': 'No comorbidity',
         'hospitalized': 'Hospitalized',
         'non_hospitalized': 'Non-hospitalized',
        'diabetes': 'Diabetes'}


def plot(obs, group_obs, plot_type='tested', correct=False):

    if plot_type == 'tested':
        blacklist = ['all', 'hospitalized', 'non_hospitalized']
    else:
        blacklist = ['all']
    fig, (ax_freq, ax_odd) = plt.subplots(1, 2, figsize=(8, 11), gridspec_kw=dict(width_ratios=(2, 1)))

    p_group_names = ['Age', 'Sex', 'Tobacco usage', 'Hospitalized']

    pos = {}
    i = 0
    yticks = []
    labels = []
    for group_name, group in groups.items():
        yticks.append(i)
        label = group_name
        if group_name in p_group_names:
            p = group_obs.loc[group_name]['p']
            if p < 0.0001:
                label += ' (p<1e-04)'
            else:
                label += f' (p={p:.4f})'
        labels.append(label)
        group_pos = i
        i -= 1.5
        for k in group:
            pos[k] = {'position': i, 'group_position': group_pos, 'group_name': group_name}
            i -= 1
        i -= 0.5

    pos = pd.DataFrame(pos).T

    pos.columns = pd.MultiIndex.from_product([['positions'], pos.columns])

    obs = obs.join(pos)


    for name, row in obs.iterrows():
        widths = np.array([row[('in', 'freq_tested')], row[('in','freq_non_tested')]])
        left = np.cumsum(widths) - widths
        ax_freq.barh(row[('positions', 'position')], widths, left=left, color=['k', 'w'], edgecolor='k', label=['Tested', 'Non tested'])
        if name not in blacklist and row[('positions', 'group_name')] not in p_group_names:
            p = row[("stat", "p")]
            if p < 0.0001:
                p = 'p<1e-04'
            else:
                p = f'p={row[("stat", "p")]:.4f}'
            ylim = 1.8 if plot_type == 'tested' else 7
            ax_odd.annotate(p, xy=(ylim, row[('positions', 'position')]), 
                            xytext=(10, 0), textcoords='offset points', xycoords='data', va='center',
                            color='gray' if row[("stat", "p")] > 0.0001 else 'k')
        centers = left + widths / 2
        for i, (x, n, w) in enumerate(zip(centers, [row[('in', 'tested')], row[('in', 'non_tested')]], widths)):
            ax_freq.annotate(f'{int(n):d}', xytext=(0, -1), xy=(x, row[('positions', 'position')]),
                        xycoords='data', textcoords='offset points', ha='center', va='center',
                        color='black' if i == 1 else 'white')
    obs_ = obs.drop(index=blacklist)
    xerr = np.abs((obs_['stat'][['odd_ratio_tested_l', 'odd_ratio_tested_u']].values - obs_['stat'][['odd_ratio_tested']].values).T)
    ax_odd.errorbar(obs_[('stat', 'odd_ratio_tested')], obs_[('positions', 'position')], xerr=xerr, zorder=100, fmt='o', capsize = 5)
    ax_odd.axvline([1], color='.5')
    ax_odd.grid(axis='y')
    ax_odd.set_yticks(obs_[('positions', 'position')])
    ax_odd.set_yticklabels([])
    ax_odd.tick_params(length=0, axis='y')

    ax_freq.xaxis.set_major_formatter(ticker.PercentFormatter(xmax=1))

    sns.despine(fig)
    ax_odd.spines['left'].set_visible(False)
    obs_no_all = obs.drop(index='all')
    ax_freq.set_yticks(obs_no_all[('positions', 'position')])
    ax_freq.set_yticklabels(obs_no_all.reset_index()['index'].replace(names))
    ax_name = fig.add_subplot(111, sharey=ax_freq)
    ax_name.set_xlim([0, 1])
    ax_freq.set_xlim([-.05, 1.05])
    ax_odd.set_ylim(ax_freq.get_ylim())
    for n, l in zip(yticks, labels):
        ax_name.annotate(l, xy=(.65, n), xycoords='data', va='center', ha='center')
    if plot_type == 'tested':
        ax_freq.set_xlabel('Proportion of tested/non-tested\n patients within each group')
        ax_odd.set_xlabel('Odds ratio of being tested \n relative to complementary group')
        ax_odd.set_xlim([0.6, 1.8])
        ax_odd.xaxis.set_major_locator(ticker.MultipleLocator(base=0.5))
        ax_odd.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.1))
        ax_odd.minorticks_on()
    else:
        ax_freq.set_xlabel('Proportion of PCR+/PCR-\ntested patients\nwithin each group')
        ax_odd.set_xlabel(f'Odds ratio of being PCR+ \nwhen tested, relative \nto complementary group')
        ax_odd.set_xscale('log')
        ax_odd.set_xticks([0.5, 1, 2, 4, 7])
        ax_odd.get_xaxis().set_major_formatter(ticker.ScalarFormatter())
        ax_odd.minorticks_on()
    ax_name.axis('off')
    fig.subplots_adjust(left=0.3, right=0.8, bottom=0.08, top=0.98)
    return fig

In [ ]:
def compute_counts(obs_test, obs_positive):
    counts_tested = obs_test['in'][['sum', 'tested', 'non_tested']].astype(int).rename(columns={'sum': 'Total', 'tested': 'Tested with PCR', 'non_tested': 'Non-tested with PCR'})
    counts_positive = obs_positive['in'][['tested', 'non_tested']].astype(int).rename(columns={'tested': 'PCR+', 'non_tested': 'PCR-'})

    counts = pd.concat([counts_tested, counts_positive], axis=1)

    counts = counts[['Total', 'Non-tested with PCR', 'Tested with PCR', 'PCR+', 'PCR-']]

    def apply(row):
        if pd.isna(row["odd_ratio_tested_l"]):
            return np.nan
        return f'[{row["odd_ratio_tested_l"]:.2f}-{row["odd_ratio_tested_u"]:.2f}]'

    odd_ratio_test = obs_test['stat'][['odd_ratio_tested_l', 'odd_ratio_tested', 'odd_ratio_tested_u']].round(2)
    odd_ratio_positive = obs_positive['stat'][['odd_ratio_tested_l', 'odd_ratio_tested', 'odd_ratio_tested_u']].round(2)

    odd_ratio_test = odd_ratio_test.apply(apply, axis=1)
    odd_ratio_positive = odd_ratio_positive.apply(apply, axis=1)

    counts = pd.concat([counts, odd_ratio_test.rename('Odd ratio of being tested').to_frame(), odd_ratio_positive.rename('Odd ratio of being PCR+').to_frame()], axis=1)
    
    def simplify(p):
        if p < 1e-4:
            return '<0.0001'
        else:
            return f'{p:.4f}'
    
    counts['Significant effect on PCR testing'] = obs_test[('stat', 'p')].apply(simplify)
    counts['Significant effect on PCR result'] = obs_positive[('stat', 'p')].apply(simplify)
    
    counts = counts.reset_index()
    counts['index'] = counts['index'].replace(names)
    counts = counts.set_index('index')

    for idx, row in counts.iterrows():
        counts.loc[idx, 'PCR+'] = f'{row["PCR+"]} ({row["PCR+"] / row["Tested with PCR"] * 100:.1f}%)'
        counts.loc[idx, 'PCR-'] = f'{row["PCR-"]} ({row["PCR-"] / row["Tested with PCR"] * 100:.1f}%)'
        counts.loc[idx, 'Tested with PCR'] = f'{row["Tested with PCR"]} ({row["Tested with PCR"] / row["Total"] * 100:.1f}%)'
        counts.loc[idx, 'Non-tested with PCR'] = f'{row["Non-tested with PCR"]} ({row["Non-tested with PCR"] / row["Total"] * 100:.1f}%)'

    
    return counts

In [ ]:
def plot_proportions(obs, group_obs):
    p_group_names = ['Age', 'Gender', 'Tobacco usage']

    pos = {}
    i = 0
    yticks = []
    labels = []

    for group_name, group in groups.items():
        yticks.append(i)
        label = group_name
        labels.append(label)
        group_pos = i
        i -= 1.5
        for k in group:
            pos[k] = {'position': i, 'group_position': group_pos, 'group_name': group_name}
            i -= 1
        i -= 0.5

    pos = pd.DataFrame(pos).T

    pos.columns = pd.MultiIndex.from_product([['positions'], pos.columns])

    obs = obs.join(pos)
    
    obs_no_all = obs.drop(index='all')

    fig, ax = plt.subplots(1, 1, figsize=(6, 11))
#     obs_ = obs.drop(index='all')
    obs_ = obs
    index = obs_.reset_index()['index'].replace(names)
    ax.grid('y')
    stat = obs_['stat']
    position = obs_['positions']
    ax.scatter(stat['freq_among_all'], position['position'], marker='d', color='C0', s=70, zorder=100, label='Full suspected\npopulation')
    ax.scatter(stat['freq_among_positive'], position['position'], marker='o', color='k', edgecolor='k', s=70, zorder=100, label='PCR+')
    ax.scatter(stat['freq_among_negative'], position['position'], marker='o', color='w', edgecolor='k', s=70, zorder=100, label='PCR-')
    ax.xaxis.set_major_formatter(ticker.PercentFormatter(xmax=1))
    ax.set_xlim([0, 1])
    ax.set_xlabel('Representation of covariate\n in supected/PCR+/PCR-/ population')
    ax.set_yticks(obs_no_all[('positions', 'position')])
    ax.set_yticklabels(obs_no_all.reset_index()['index'].replace(names))
    for n, l in zip(yticks, labels):
        ax.annotate(l, xy=(.5, n), xycoords='data', va='center', ha='center')
    ax.legend(frameon=True, loc='lower left', bbox_to_anchor= (0.7, 0.2))
    sns.despine(fig)
    fig.subplots_adjust(left=0.3, right=0.8, bottom=0.08, top=0.98)
    return fig

In [ ]:
for correct in [True, False]:
    correct_str = "_correct" if correct else ""
    obs_test, group_obs_test = make_observation_df(merged, 'test_done', correct=correct)
    obs_test.to_csv(f'output/cohort_test{correct_str}.csv')
    fig = plot(obs_test, group_obs_test, plot_type='tested', correct=correct)
    fig.savefig(f'output/tested_cohort{correct_str}.pdf')
    obs_positive, group_obs_positive = make_observation_df(merged.loc[merged['test_done']], 'pcr_results', correct=correct)
    obs_positive.to_csv(f'output/cohort_positive{correct_str}.csv')  # Note positive means tested in this case
    fig = plot(obs_positive, group_obs_positive, plot_type='positive', correct=correct)
    fig.savefig(f'output/positive_cohort{correct_str}.pdf')
    counts = compute_counts(obs_test, obs_positive)
    counts.to_csv(f'output/count_{correct_str}')
    fig = plot_proportions(obs_test, group_obs_test)
    fig.savefig(f'output/pcr_representation{correct_str}.pdf')
    obs_hospitalized, group_obs_hospitalized = make_observation_df(merged.loc[~merged['hospitalized'] & merged['test_done']], 'pcr_results', correct=correct)
    fig = plot(obs_hospitalized, group_obs_hospitalized, plot_type='positive', correct=correct)
    fig.savefig(f'output/positive_non_hospitalized_cohort{correct_str}.pdf')

## BIG CSV

In [ ]:
csv_test = obs_test[['in', 'stat']].round(2)
csv_test.columns = pd.Index(['Tested', 'Not tested', 'All', 'Tested freq', 'Non-tested freq', 'Group size among all', 'Group size among positive', 'Group size among negative', 'Tested odd ratio (lower)', 'Tested odd ratio', 'Tested odd ratio (upper)', 'p-value: influence on PCR access'])
csv_test['p-value: influence on PCR access'] = obs_test[('stat', 'p')]

In [ ]:
csv_positive = obs_positive[['in', 'stat']].drop(columns=[('stat', 'freq_among_all'), ('stat', 'freq_among_positive'), ('stat', 'freq_among_negative')]).round(2)
csv_positive.columns = pd.Index(['Positive', 'Negative', 'Tested', 'Positive freq', 'Negative freq', 'Positive odd ratio (lower)', 'Positive odd ratio', 'Positive odd ratio (upper)', 'p-value: influence on PCR positivity'])
csv_positive['p-value: influence on PCR positivity'] = obs_positive[('stat', 'p')]

In [ ]:
csv = pd.concat([csv_test, csv_positive.drop(columns='Tested')], axis=1)

In [ ]:
csv = csv.reset_index()
csv['index'] = csv['index'].replace(names)
csv = csv.set_index('index')

In [ ]:
csv.to_csv('output/statistics.csv')

In [ ]:
fig = plot_proportions(obs_test, group_obs_test)
fig.savefig('output/pcr_representation.pdf')

In [ ]:
positive = merged.loc[merged['pcr_results'].fillna(False)]
negative = merged.loc[~(merged['pcr_results'].fillna(True))]

In [ ]:
positive['start_reason'].value_counts() / positive['start_reason'].count()

## Paper numbers

In [ ]:
def count_freq(x):
    return pd.DataFrame({'count': x.value_counts(), 'freq':x.value_counts() / x.count()})

In [ ]:
merged['ageusia&anosmia'] = merged['ageusia'] & merged['anosmia']
merged['ageusia|anosmia'] = merged['ageusia'] | merged['anosmia']

In [ ]:
merged.groupby(by='gender')['ageusia&anosmia'].apply(count_freq)

In [ ]:
merged.groupby(by='ageusia&anosmia')['age'].aggregate(['mean', 'std'])

In [ ]:
merged.groupby(by='ageusia|anosmia')['hospitalized'].apply(count_freq)

In [ ]:
y = merged['hospitalized'].astype(float)
X = merged['ageusia|anosmia'].astype(float)
X = sm.add_constant(X)
logit_mod = sm.Logit(y, X)
logit_res = logit_mod.fit()
mean = logit_res.params['ageusia|anosmia']
se = logit_res.bse['ageusia|anosmia']
p = logit_res.pvalues['ageusia|anosmia']

or_ = np.exp(np.array([mean - se, mean + se]))

or_, p

In [ ]:
merged['to_ctscan'] = ~(merged['ageusia'] | merged['anosmia']) & merged['cardiopulmonary']

merged.groupby(by='to_ctscan')['ctscan'].apply(lambda x: pd.DataFrame({'count': x.value_counts(), 'freq':x.value_counts() / x.count()}))
merged.groupby(by='to_ctscan')['ctscan'].apply(lambda x: pd.DataFrame({'count': x.value_counts(), 'freq':x.value_counts() / x.count()}))

In [ ]:
merged['start_reason'].value_counts()